I. Towards writing clear instructions

1. Towards a concise and detailed query.

First prompt (simple) in English and Darija. Input is in Darija.


In [5]:
from openai import OpenAI

client = OpenAI()

def translate_verb(verb):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {
                "role": "system",
                "content": "غادي نعطيك واحد الفعل بالدارجة المغربية، والمهمة ديالك هي تترجمو لنجليزية."
            },
            {
                "role": "user", 
                "content": f"{verb}"
            }
        ],
        temperature=0
    )
    return completion.choices[0].message.content

def translate_verb1(verb):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {
                "role": "system",
                "content": "You will be provided with a verb in Moroccan Darija language, and your task is to translate it into English."
            },
            {
                "role": "user", 
                "content": 'Verb: داق'
            }
                ],
        temperature=0
    )
    return completion.choices[0].message.content

 # max_tokens=1, # in case of a dot
 # top_p=0.1
 # temperature: What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
 # top_p:An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
 # max_tokens: The maximum number of tokens that can be generated in the chat completion.

# to be included in report

print(translate_verb('داق'))
print(translate_verb1('داق'))


"داق" بالدارجة المغربية تعني "to taste" بالإنجليزية.
To taste


In [ ]:
from openai import OpenAI

client = OpenAI()

test = client.chat.completions.create(
model =  "gpt-3.5-turbo",
messages = [
            {"role": "system", "content": "نتا خبير فلهجة الدارجة المغربية وفالترجمة من الدارجة لنجليزية."},
            {"role": "user", "content": "ترجم الفعل المعطي بالدارجة المغربية لنجليزية كلمة بكلمة." + 
             "مخصش يكون المخرج فعل مصدري. بلاما تعطي تفسيرات."},
            {"role": "assistant", "content": "واخا. فهمت. أنا مستعد للفعل"},
            {"role": "user", "content": f"{verb}"},
],
temperature=0, # deterministic
)

test2 = client.chat.completions.create(
model =  "gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Moroccan Darija dialect and in translating Darija to English."},
            {"role": "user", "content": "Translate the provided Moroccan Darija dialect verb into English using word-by-word translation." + 
             "The output must not be an infinitive verb. No explanations are needed in the output"},
            {"role": "assistant", "content": "OK, I understand. I am ready for the verb translation."},
            {"role": "user", "content": f"{verb}"},
        ],
temperature=0, 

test = test.choices[0].message.content
print(test)
test2 = test2.choices[0].message.content
print(test2)

In [ ]:
from openai import OpenAI
import pandas as pd
client = OpenAI()
df = pd.read_csv('sentences.csv')


def arb_prmpt_translate(verb):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
    {
        "role": "system",
        "content": "You will be provided with a verb in Moroccan Darija language." + 
        "Your task is to translate it into English and provide only its lemma as an output." + 
                    "The lemma of the translated verb should not contain 'to'."
    },
    {
        "role": "user", 
        "content":  f'{verb}' 
    }
            ], 
        temperature=0.1  
        )
    return completion.choices[0].message.content

def eng_prmpt_translate(verb):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
    {
        "role": "system",
        "content": "You will be provided with a verb in Moroccan Darija language." + 
        "Your task is to translate it into English and provide only its lemma as an output." + 
                    "The lemma of the translated verb should not contain 'to'."
    },
    {
        "role": "user", 
        "content":  f'{verb}' 
    }
            ], 
        temperature=0.1  
        # Deterministic responses
        # max_tokens=1, # in case of a dot
        )
    return completion.choices[0].message.content
df['transl'] = df['n1'].apply(translate)

print(df['transl'])

from nltk.corpus import wordnet as wn


transl = df['transl']
eng = df['eng']
transl = [verb.replace(" ", "_") for verb in transl]
eng = [verb.replace(" ", "_") for verb in eng]

def max_path_similarity(word1, word2):
    """Calculate the maximum path similarity between two words."""
    synsets1 = wn.synsets(word1, pos=wn.VERB)
    synsets2 = wn.synsets(word2, pos=wn.VERB)
    max_score = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            score = syn1.path_similarity(syn2)
            if score and score > max_score:
                max_score = score
    return max_score
# Calculate and print the path similarity for each pair of verbs with similar index

def evaluate_pairs(threshold=0.5):
    """Evaluating each pair using WordNet path similarity and a threshold."""
    results = []
    for i in range(len(transl)):
        word1, word2 = transl[i], eng[i]
        similarity = max_path_similarity(word1, word2)
        results.append((i+2, word1, word2, similarity, similarity >= threshold))
    return results
results = evaluate_pairs()
print(results)
import matplotlib.pyplot as plt

similarities = [item[3] for item in results]

# Calculate the average similarity
average_similarity = sum(similarities) / len(similarities)

# Print the result
print(f'Average similarity: {average_similarity:.2f}%')


# Define intervals of 10%
intervals = [(i, i+10) for i in range(0, 90, 10)] + [(90, 101)]

# Count similarities in each interval
interval_counts = {interval: 0 for interval in intervals}
for sim in similarities:
    percentage = sim * 100  # Convert cosine similarity to percentage
    for interval in intervals:
        if interval[0] <= percentage < interval[1]:
            interval_counts[interval] += 1
            break



# Count pairs in the interval 90-100%
count_90_100 = interval_counts[(90, 101)]

# Print the count for the interval 90-100%
print(f'Count of pairs in the interval 90-100%: {count_90_100}')



# Prepare data for plotting
labels = [f'{interval[0]}-{interval[1]-1}' if interval[1] != 101 else '90-100' for interval in intervals]
counts = [interval_counts[interval] for interval in intervals]

# Plot bar chart
plt.bar(labels, counts, color='skyblue')
plt.xlabel('Similarity (%)')
plt.ylabel('Count of pair verbs')
plt.title('Similarity per interval')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

chack previous work with stratified 

should the transliteration be any more checked within itself between the two results?

there must be a clearance of all useless work.

work from now on is with average. Makes sense than threshold.